In [39]:
# url
url = 'https://dakar-auto.com/senegal/motos-and-scooters-3?&page=3'
# Assignment instructions are:
#scrape the following informations:
#brand, model, year,reference, kms driven, adress, price, img_link
# from page 1 to 200
#clean the scraped data

In [40]:
import pandas as pd
from requests import get  # it allows to get the content (html) of webpage
from bs4 import BeautifulSoup as bs # store the content in beautiful in beautifulSoup object and sparse the code

In [41]:
url = 'https://dakar-auto.com/senegal/motos-and-scooters-3?&page=3'
# get the html code of the page using the get function requests
res = get(url)
# store the html code in a beautifulsoup objet with a html parser (a parser allows to easily navigate through the html code)
soup = bs(res.content, 'html.parser' )
print(soup)


<!DOCTYPE html>

<html class="no-js" lang="fr-SN">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="fr" http-equiv="content-language"/>
<title>Motos &amp; scooters occasion ou neuf Sénégal - Achat et vente motos &amp; scooters d'occasion ou neuf, annonce gratuite - Dakar-Auto.com   - Page 3</title>
<meta content="FYo5KU8AIHZPWvujO_6041XfouWqt_ScJvfMH1afIEo" name="google-site-verification">
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://dakar-auto.com/senegal/motos-and-scooters-3?&amp;page=3" rel="canonical"/>
<meta content="Expat, résident, professionnel, parcoure notre liste de moto &amp; scooters d'occasion et neuf et faite une affaire dès aujourd'hui, pour l'achat ou la vente de votre moto ou scooter d'occasion au Sénégal, choisissez Dakar-Auto.com, site gratuit d'annonce automobile, moto et scooter.    Page 3" name="description">
<meta content="Expat, résident, professionnel, pour l'acha

In [42]:
# get all containers that contains the informations of each motos and sooter
containers = soup.find_all('div', class_ = "listings-cards__list-item mb-md-3 mb-3")
# length of containers
len(containers)

20

In [43]:
# But we need to make sure the container we choose contain all the informations we need
container = containers[0]
#print(container)

In [46]:
#scrape the following informations:
#brand, model, year,reference, kms driven, adress, price
gen_inf = container.find('h2','listing-card__header__title mb-md-2 mb-0').a.text.strip().split()
# get the brand
brand = gen_inf[0]
#get the model
model = " ".join(gen_inf[1:len(gen_inf)-1])
# get the year
year = gen_inf[-1]
# Scrape the reference, kilometer driven, fuel type and gearbox type
gen_inf1 = container.find_all('li','listing-card__attribute list-inline-item')
# get reference
reference = gen_inf1[0].text.split()[-1]
# get kilometer driven
km_length = gen_inf1[1].text.replace('km','')
# to get adress
address ="".join(container.find('div','col-12 entry-zone-address').text.strip().split())
price = "".join(container.find('h3','listing-card__header__price font-weight-bold text-uppercase mb-0').text.strip().split()).replace('FCFA', '')

# get the image link
img = container.find('img')
img_link = 'https://dakar-auto.com'  + img['src']
# print all the infos for the first container


In [47]:
data =[]
for container in containers:
  try:
      gen_inf = container.find('h2','listing-card__header__title mb-md-2 mb-0').a.text.strip().split()
      # get the brand
      brand = gen_inf[0]
      #get the model
      model = " ".join(gen_inf[1:len(gen_inf)-1])
      # get the year
      year = gen_inf[-1]
      # Scrape the reference, kilometer driven, fuel type and gearbox type
      gen_inf1 = container.find_all('li','listing-card__attribute list-inline-item')
      # get reference
      reference = gen_inf1[0].text.split()[-1]
      # get kilometer driven
      km_length = gen_inf1[1].text.replace('km','').strip()
      # to get adress
      address ="".join(container.find('div','col-12 entry-zone-address').text.strip().split())
      price = "".join(container.find('h3','listing-card__header__price font-weight-bold text-uppercase mb-0').text.strip().split()).replace('FCFA', '')
      # get image link
      img = container.find('img')
      img_link = 'https://dakar-auto.com'  + img['src']
      # print all the infos for the first container
      dic = {
            "brand":brand,
            "model":model,
            "year":year,
            "reference":reference,
            "km_length":km_length,
            "address":address,
            "price":price
        }
      data.append(dic)
  except:
    pass
DF = pd.DataFrame(data)

In [49]:
# data of the page
DF.head()

,brand,model,year,reference,km_length,address,price
0,KTM,jakarta,2021,138883,1000,"Point-E,Dakar",350000
1,KTM,SX,2015,138870,1,"Guédiawaye,Dakar",1400000
2,Yamaha,TMax,2012,138797,101000,"VDN,Dakar",2350000
3,Yamaha,TMax,2012,138800,98000,"VDN,Dakar",2300000
4,Yamaha,TMax,2012,138801,98000,"VDN,Dakar",2350000


### <font color = 'tomato'> To Collect data over multiple pages

In [50]:
# To create a empty dataframe df
df = pd.DataFrame()
# loop over pages indexes
for index in range(1, 201):
  url = f'https://dakar-auto.com/senegal/motos-and-scooters-3?&page={index}'
  # get the html code of the page using the get function requests
  res = get(url)
  # store the html code in a beautifulsoup objet with a html parser (a parser allows to easily navigate through the html code)
  soup = bs(res.content, 'html.parser' )
  # get all containers that contains the informations of each motos and sooter
  containers = soup.find_all('div', class_ = "listings-cards__list-item mb-md-3 mb-3")
  # scrape data from all the containers
  data = []
  for container in containers:
    try:
      gen_inf = container.find('h2','listing-card__header__title mb-md-2 mb-0').a.text.strip().split()
      # get the brand
      brand = gen_inf[0]
      # To get the model
      model = " ".join(gen_inf[1:len(gen_inf)-1])
      # get the year
      year = gen_inf[-1]
      # Scrape the reference, kilometer driven, fuel type and gearbox type
      gen_inf1 = container.find_all('li','listing-card__attribute list-inline-item')
      # get reference
      reference = gen_inf1[0].text.split()[-1]
      # get kilometer driven
      km_length = gen_inf1[1].text.replace('km','').strip()
      # to get adress
      address ="".join(container.find('div','col-12 entry-zone-address').text.strip().split())
      price = "".join(container.find('h3','listing-card__header__price font-weight-bold text-uppercase mb-0').text.strip().split()).replace('FCFA', '')
      # get the image link
      img = container.find('img')
      img_link = 'https://dakar-auto.com'  + img['src']
      # print all the infos for the first container
      dic = {
            "brand":brand,
            "model":model,
            "year":year,
            "reference":reference,
            "km_length":km_length,
            "address":address,
            "price":price
        }
      data.append(dic)
    except:
      pass

  DF = pd.DataFrame(data)
  df = pd.concat([df, DF], axis = 0).reset_index(drop = True)

In [11]:
df.head()

,brand,model,year,reference,km_length,address,price
0,Yamaha,TMax,2023,139869,\n\n1,"VDN,Dakar",4300000
1,Yamaha,X-Max,2025,139807,\n\n250,"SicapBaobab,Dakar",800000
2,Honda,X-ADV,2021,139806,\n\n350,"Cambérène,Dakar",810000
3,KTM,jakarta,2025,139757,\n\n6000,"VDN,Dakar",220000
4,Yamaha,Tricker,2025,138849,\n\n1,"SicapSacréCoeur,Dakar",950000


In [12]:
# To get dimension
df.shape

(506, 7)

In [52]:
# To get data types
df.dtypes

,0
brand,object
model,object
year,object
reference,object
km_length,object
address,object
price,object


In [51]:
# To get describe the dataframe
df.describe()

,brand,model,year,reference,km_length,address,price
count,506,506,506,506,506,506,506
unique,26,230,37,501,199,58,120
top,Yamaha,X-Max,2010,133329,1,"Grand-Dakar,Dakar",1600000
freq,123,18,47,2,64,298,21


In [22]:
# To get information of the DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   brand      506 non-null    object
 1   model      506 non-null    object
 2   year       506 non-null    object
 3   reference  506 non-null    object
 4   km_length  506 non-null    object
 5   address    506 non-null    object
 6   price      506 non-null    object
dtypes: object(7)
memory usage: 27.8+ KB


In [71]:
df.year = pd.to_numeric(df.year, errors='coerce').astype('Int64')
df.price = df.price.replace('NC', np.nan)
df.price = pd.to_numeric(df.price, errors='coerce').astype('Int64')
df.km_length = df.km_length.astype('float64')
df.km_length = pd.to_numeric(df.km_length, errors = "coerce")

df.dtypes

,0
brand,object
model,object
year,Int64
reference,object
km_length,float64
address,object
price,Int64


In [72]:
# check for duplicated values
df.duplicated().any().sum()

np.int64(0)

In [74]:
# check for null values
print(df.isnull().any().sum())

2


In [76]:
df.dropna(inplace=True)
df.head()

,brand,model,year,reference,km_length,address,price
0,Yamaha,TMax,2023,139869,1.0,"VDN,Dakar",4300000
1,Yamaha,X-Max,2025,139807,250.0,"SicapBaobab,Dakar",800000
2,Honda,X-ADV,2021,139806,350.0,"Cambérène,Dakar",810000
3,KTM,jakarta,2025,139757,6000.0,"VDN,Dakar",220000
4,Yamaha,Tricker,2025,138849,1.0,"SicapSacréCoeur,Dakar",950000


In [78]:
# check for null values
print(df.isnull().any().sum())

0
